# Identifying arXiv Article Subject Codes via NLP

The goal of this project is to predict primary subject codes for scientific articles available in the arXiv database based on the text of their abstract. This allows for rapid encoding of article subject material, and similar methods may be applicable for identifying key terms in articles submitted for addition to the database. 

Without machine learning, idexing articles for addition to a database along subject codes, key terms, and other metrics is a labor intensive process. For some databases, the labor cost of indexing a single article has been estimated to cost up to 10 dollars per article. Natural Language Processing offers the ability to automate this process, thereby saving up to 15 million dollars in labor for a dataset similar in size to the one used for this project upon intial upload. Maintenance costs may see further cost savings as routine updates to the database architecture or indexing system can be automated instead of requiring large quantities of expensive manual labor.

In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.feature_extraction import FeatureHasher

import nltk

In [2]:
#import data
df = pd.read_csv("./arxiv-oai-af.tsv", delimiter="\t")
df

,abstract,acm_class,arxiv_id,author_text,categories,comments,created,doi,num_authors,num_categories,primary_cat,title,updated
0,If we assume the Thesis that any classical T...,NaN,math/0212388,Bhupinder Singh Anand,math.GM,12 pages. Revision 1. Appendix 1 added. An HTM...,2002-12-31,NaN,1,1,math.GM,Is a deterministic universe logically consiste...,2003-01-02
1,"We define the Cartesian product, composition...",NaN,1205.6123,"Muhammad Akram, Wieslaw A. Dudek",cs.DM,NaN,2012-04-29,10.1016/j.camwa.2010.11.004,2,1,cs.DM,Interval-valued fuzzy graphs,NaN
2,We apply algebraic Morse theory to the Taylo...,NaN,1806.07887,Robin Frankhuizen,"math.AT,math.AC,math.RA",27 pages; comments welcome. arXiv admin note: ...,2018-06-20,NaN,1,3,math.AT,Massey products and the Golod property for sim...,NaN
3,Anomalous transport is usually described eit...,NaN,1007.3022,"Bartlomiej Dybiec, Ewa Gudowska-Nowak","cond-mat.stat-mech,math-ph,math.MP","10 pages, 7 figures",2010-07-18,10.1063/1.3522761,2,3,cond-mat.stat-mech,Subordinated diffusion and CTRW asymptotics,2010-11-09
4,"In this paper, an approximate solution to a ...",NaN,1512.07787,"M. T. Araujo, E. Drigo Filho",cond-mat.stat-mech,"12 pages, 8 figures",2015-12-24,10.5488/CMP.18.43003,2,1,cond-mat.stat-mech,Approximate solution for Fokker-Planck equation,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582236,21cm intensity mapping experiments aim to ob...,NaN,1501.03823,"Laura Wolz, Filipe B. Abdalla, David Alonso, C...",astro-ph.CO,This article is part of the 'SKA Cosmology Cha...,2015-01-15,NaN,10,1,astro-ph.CO,Foreground Subtraction in Intensity Mapping wi...,NaN
1582237,We show the existence of smooth isolated cur...,NaN,math/0110220,Andreas Leopold Knutsen,math.AG,18 pages. The previous version of the preprint...,2001-10-19,NaN,1,1,math.AG,"Smooth, isolated curves in families of Calabi-...",2012-09-05
1582238,Sequence alignment is a tool in bioinformati...,NaN,0907.2187,"S Wolfsheimer, O Melchert, AK Hartmann","cond-mat.stat-mech,cond-mat.dis-nn,q-bio.QM",NaN,2009-07-13,10.1103/PhysRevE.80.061913,3,3,cond-mat.stat-mech,Finite-temperature local protein sequence alig...,NaN
1582239,"We suggest that the majority of the ""young"",...",NaN,astro-ph/0209553,Valery V. Kravtsov,astro-ph,"7 pages, no figures, accepted for publication ...",2002-09-26,10.1051/0004-6361:20021404,1,1,astro-ph,Second Parameter Globulars and Dwarf Spheroida...,NaN


In [3]:
#find explicit nulls
df.isnull().sum()

abstract                0
acm_class         1560822
arxiv_id                0
author_text             0
categories              0
comments           301829
created                 0
doi                734897
num_authors             0
num_categories          0
primary_cat             0
title                   0
updated            991881
dtype: int64

In [4]:
#drop unnecessary columns 
trimmed_df = df.drop(columns=["acm_class", 
                              "comments", 
                              "created", 
                              "num_authors", 
                              "num_categories", 
                              "updated", 
                              "doi", 
                              "categories", 
                              "author_text", 
                              "title"]
                    )
trimmed_df

,abstract,arxiv_id,primary_cat
0,If we assume the Thesis that any classical T...,math/0212388,math.GM
1,"We define the Cartesian product, composition...",1205.6123,cs.DM
2,We apply algebraic Morse theory to the Taylo...,1806.07887,math.AT
3,Anomalous transport is usually described eit...,1007.3022,cond-mat.stat-mech
4,"In this paper, an approximate solution to a ...",1512.07787,cond-mat.stat-mech
...,...,...,...
1582236,21cm intensity mapping experiments aim to ob...,1501.03823,astro-ph.CO
1582237,We show the existence of smooth isolated cur...,math/0110220,math.AG
1582238,Sequence alignment is a tool in bioinformati...,0907.2187,cond-mat.stat-mech
1582239,"We suggest that the majority of the ""young"",...",astro-ph/0209553,astro-ph


In [5]:
#check for duplicate articles
trimmed_df["arxiv_id"].value_counts()

hep-th/0508125      1
astro-ph/0210065    1
0909.2631           1
1801.05503          1
1212.0748           1
                   ..
0905.3520           1
1406.3435           1
1908.00251          1
1003.6128           1
gr-qc/9906088       1
Name: arxiv_id, Length: 1582241, dtype: int64

In [6]:
#full dataset turned out to be too large for training with available resources
#looking for best training subset based on class balance
trimmed_df["primary_cat"].value_counts()[:50]

hep-ph                107925
astro-ph               94239
hep-th                 86019
quant-ph               71567
cond-mat.mes-hall      46310
gr-qc                  45681
cond-mat.mtrl-sci      39348
cond-mat.str-el        35899
cond-mat.stat-mech     32370
astro-ph.SR            30147
astro-ph.CO            29493
math.AP                28783
math.CO                27755
nucl-th                27524
astro-ph.GA            27103
math.PR                26395
cs.CV                  25941
math-ph                25426
math.AG                25312
cond-mat.supr-con      25137
astro-ph.HE            24153
cs.IT                  23048
math.NT                20834
math.DG                20613
cond-mat.soft          19548
hep-ex                 18774
cs.LG                  18178
physics.optics         17130
hep-lat                15185
math.OC                14899
math.DS                14742
math.NA                13912
math.FA                12971
astro-ph.EP            12844
cond-mat      

In [7]:
#taking top 10 most frequent subjects
n_subjects = 10
#identify the low frequency subjects
balance = trimmed_df["primary_cat"].value_counts()[:n_subjects]
index = np.array(balance.index)

#drop low frequency subjects
subject = trimmed_df["primary_cat"]
balanced_df = trimmed_df[subject.isin(index)]
balanced_df.shape

(589505, 3)

Train-test split

In [8]:
train, test = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [9]:
#save data for later
test.to_csv("./Train&Test/test_split.csv")
train.to_csv("./Train&Test/train_split.csv")

### Model Construction

The general approach is to create a neural network that will encode the data from the abstract, train on those abstracts using its recurrent architecture, and then return the subject codes that correspond. Two methods are generally used for this: LSTM and GRU architecture. The initial model will use GRU architecture due to computational limitations and the size of the dataset, but both mechanisms are candidates for this model.

In [10]:
#target encoder
def encode_target(data, features=n_subjects, input_type="string"):
    """takes a set of y values and one hot encodes them for the Neural Network output"""
    FH = FeatureHasher(n_features=features, input_type=input_type)
    target = FH.fit_transform(X=data)
    target_array = target.toarray()
    return target_array

In [11]:
#one hot encoding target for training set
train_target = encode_target(train["primary_cat"])

In [12]:
#creating encoder to clean and encode abstract data
encoder = layers.experimental.preprocessing.TextVectorization(output_mode='int')
#calling adapt gets the layer to index all of the terms
#this step speeds up model performance and reduces parameters
encoder.adapt(np.array(train["abstract"]))

Exploding gradient turned out to be a significant issue when constructing the initial model. In order to address this problem, several modifications were made during training and the number of subject headings was limited to a subset of the 10 most common headings in order to complete a few epochs and collect data that could help diagnose the problem. Gradient clipping was implemented quickly, but gradient instability persists.

In [17]:
#initial model
init_model = Sequential([
    #input layer
    tf.keras.Input(shape=(1,), dtype=tf.string),
    #encoder from cell above
    encoder,
    #embedding layer
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary())+1,
        output_dim=10,
        # Use masking to handle the variable abstract lengths
        mask_zero=True),
    #recurrent layer
    tf.keras.layers.GRU(32),
    #decision time
    tf.keras.layers.Dense(32, activation='tanh'),
    tf.keras.layers.Dense(n_subjects, activation='softmax')
])

#compiling the model
init_model.compile(
    loss='categorical_hinge',
    #clipping due to initial issues with exploding gradient
    optimizer=tf.keras.optimizers.Adam(clipnorm=1.0),
    metrics=['accuracy', 
             'AUC', 
             tf.keras.metrics.CategoricalCrossentropy()]
)

#fit the model
print(init_model.summary())
NL1 = init_model.fit(x=train["abstract"], y=train_target, epochs=10,
                     batch_size=32, validation_split=0.3)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 10)          5565770   
_________________________________________________________________
gru_4 (GRU)                  (None, 32)                4224      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                330       
Total params: 5,571,380
Trainable params: 5,571,380
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
10317/10317 [==============================] - 1524s 148ms/step - loss: 0.6281 - accuracy: 0.5

In [23]:
#save the initial model
tf.saved_model.save(init_model, "./models/")

INFO:tensorflow:Assets written to: ./models/assets


### Model Iteration

### Model Evaluation

# Discussion